<a href="https://colab.research.google.com/github/rishabhpurohit/TransferLearning-ai1-fall2019/blob/master/pretraining%20error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!ls


dogcat-data  dogcat-data.tgz  sample_data  wandb


In [4]:
!pwd

/content


In [5]:
!wget https://storage.googleapis.com/wandb-production.appspot.com/qualcomm/dogcat-data.tgz

--2019-09-29 13:29:36--  https://storage.googleapis.com/wandb-production.appspot.com/qualcomm/dogcat-data.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84807750 (81M) [application/gzip]
Saving to: ‘dogcat-data.tgz.1’

dogcat-data.tgz.1   100%[===================>]  80.88M   116MB/s    in 0.7s    

2019-09-29 13:29:37 (116 MB/s) - ‘dogcat-data.tgz.1’ saved [84807750/84807750]



In [6]:
!tar xvzf dogcat-data.tgz

dogcat-data/
dogcat-data/train/
dogcat-data/validation/
dogcat-data/validation/cat/
dogcat-data/validation/dog/
dogcat-data/validation/dog/._dog.1753.jpg
dogcat-data/validation/dog/dog.1753.jpg
dogcat-data/validation/dog/._dog.1035.jpg
dogcat-data/validation/dog/dog.1035.jpg
dogcat-data/validation/dog/._dog.1021.jpg
dogcat-data/validation/dog/dog.1021.jpg
dogcat-data/validation/dog/._dog.1747.jpg
dogcat-data/validation/dog/dog.1747.jpg
dogcat-data/validation/dog/._dog.1009.jpg
dogcat-data/validation/dog/dog.1009.jpg
dogcat-data/validation/dog/._dog.1790.jpg
dogcat-data/validation/dog/dog.1790.jpg
dogcat-data/validation/dog/._dog.1948.jpg
dogcat-data/validation/dog/dog.1948.jpg
dogcat-data/validation/dog/._dog.1784.jpg
dogcat-data/validation/dog/dog.1784.jpg
dogcat-data/validation/dog/._dog.1974.jpg
dogcat-data/validation/dog/dog.1974.jpg
dogcat-data/validation/dog/._dog.1960.jpg
dogcat-data/validation/dog/dog.1960.jpg
dogcat-data/validation/dog/._dog.1237.jpg
dogcat-data/validation/dog

In [0]:
!rm dogcat-data/train/dog/._dog*

In [0]:
!rm dogcat-data/train/cat/._cat*

In [9]:
!ls dogcat-data

train  validation


In [0]:
!rm dogcat-data/validation/dog/._dog*

In [0]:
!rm dogcat-data/validation/cat/._cat*

In [12]:
!ls dogcat-data/train

cat  dog


In [13]:
ls -1 dogcat-data/train/dog/ | wc -1

wc: invalid option -- '1'
Try 'wc --help' for more information.


In [14]:
!ls -1 dogcat-data/validation/dog/ | wc -1

wc: invalid option -- '1'
Try 'wc --help' for more information.


In [15]:
pip install wandb

In [16]:
import wandb
wandb.init()

W&B Run: https://app.wandb.ai/rishabh/uncategorized/runs/kavuxzyf

In [22]:
!wget https://raw.githubusercontent.com/rishabhpurohit/TransferLearning-ai1-fall2019/master/dogcat_data.py

--2019-09-29 13:36:31--  https://raw.githubusercontent.com/rishabhpurohit/TransferLearning-ai1-fall2019/master/dogcat_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1498 (1.5K) [text/plain]
Saving to: ‘dogcat_data.py’

dogcat_data.py      100%[===================>]   1.46K  --.-KB/s    in 0s      

2019-09-29 13:36:31 (242 MB/s) - ‘dogcat_data.py’ saved [1498/1498]



In [0]:
import numpy as np
#from keras.preprocessing import Sequence
# import keras.utils
# keras.utils.Sequence = Sequence
from keras.preprocessing.image import ImageDataGenerator
#from myimage import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import Callback
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16, preprocess_input
from dogcat_data import generators, get_nb_files
import os
import sys
import wandb
from wandb.keras import WandbCallback

In [46]:
wandb.init()
config = wandb.config

# dimensions of our images.
config.img_width = 224
config.img_height = 224
config.epochs = 50
config.batch_size = 40

top_model_weights_path = 'bottleneck.h5'
train_dir = 'dogcat-data/train'
validation_dir = 'dogcat-data/validation'
nb_train_samples = 1000
nb_validation_samples = 1000


if os.path.exists('bottleneck_features_train.npy') and (len(sys.argv) == 1 or sys.argv[1] != "--force"):
 print("Using saved features, pass --force to save new features")
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = datagen.flow_from_directory(
  train_dir,
  target_size=(config.img_width, config.img_height),
  batch_size=config.batch_size,
  class_mode="binary")

val_generator = datagen.flow_from_directory(
  validation_dir,
  target_size=(config.img_width, config.img_height),
  batch_size=config.batch_size,
  class_mode="binary")

    




Found 1800 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [47]:
# build the VGG16 network
    model = VGG16(include_top=False, weights='imagenet')

    print("Predicting bottleneck training features")
    training_labels = []
    training_features = []
    for batch in range(nb_train_samples // config.batch_size):
        data, labels = next(train_generator)
        training_labels.append(labels)
        training_features.append(model.predict(data))
    

Predicting bottleneck training features
Predicting bottleneck validation features


In [56]:
training_labels

array([0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
       1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
       1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
       0., 0., 1., 0., 0.

In [55]:
training_labels = np.concatenate(training_labels)
training_features = np.concatenate(training_features)
np.savez(open('bottleneck_features_train.npy', 'wb'),
  features=training_features, labels=training_labels)

print("Predicting bottleneck validation features")
validation_labels = []
validation_features = []
validation_data = []
for batch in range(nb_validation_samples // config.batch_size):
    data, labels = next(val_generator)
    validation_features.append(model.predict(data))
    validation_labels.append(labels)
    validation_data.append(data)
validation_labels = np.concatenate(validation_labels)
validation_features = np.concatenate(validation_features)
validation_data = np.concatenate(validation_data)
np.savez(open('bottleneck_features_validation.npy', 'wb'),
  features=validation_features, labels=validation_labels, data=validation_data)

ValueError: ignored

In [0]:
def train_top_model():
    train = np.load(open('bottleneck_features_train.npy', 'rb'))
    X_train, y_train = (train['features'], train['labels'])
    test = np.load(open('bottleneck_features_validation.npy', 'rb'))
    X_test, y_test, val_data = (test['features'], test['labels'], test['data'])

    model = Sequential()
    model.add(Flatten(input_shape=X_train[0].shape))
    model.add(Dense(256, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    class Images(Callback):
        def on_epoch_end(self, epoch, logs):
            base_model = VGG16(include_top=False, weights='imagenet')
            indices = np.random.randint(val_data.shape[0], size=36)
            test_data = val_data[indices]
            features = base_model.predict(
                np.array([preprocess_input(data) for data in test_data]))
            pred_data = model.predict(features)
            wandb.log({
                "examples": [
                      wandb.Image(
                          test_data[i], caption="cat" if pred_data[i] < 0.5 else "dog")
                      for i, data in enumerate(test_data)]
            }, commit=False)

    model.fit(X_train, y_train,
              epochs=config.epochs,
              batch_size=config.batch_size,
              validation_data=(X_test, y_test),
              callbacks=[Images(), WandbCallback(save_model=False)])
    model.save_weights(top_model_weights_path)




In [0]:
save_bottlebeck_features()
train_top_model()